In [1]:
import gzip
import json
import ast
import pandas as pd


# Carga y lectura del archivo en formato json.gz y conversion del archivo a un dataframe
archivo_gz_user_items = r"C:\Users\veram\OneDrive\Escritorio\mi_directorio\Datos.gz\users_items.json.gz"

data_items = []
with gzip.open(archivo_gz_user_items, 'rt', encoding='utf-8') as file:
    for line in file.readlines():
        data_items.append(ast.literal_eval(line))

df_items  = pd.DataFrame(data_items)
df_items_clean = df_items.copy()



In [2]:
# aplicar "explode" a la columna 'reviews' 
items_exploded = df_items_clean.explode('items').reset_index(drop=True)

#Convertir los datos anidados  de la columna 'reviews' a columnas separadas
items_expanded = pd.json_normalize(items_exploded['items'])
data_final_items = pd.concat([items_exploded.drop(columns=['items']), items_expanded], axis=1)

print(data_final_items)

                   user_id  items_count           steam_id  \
0        76561197970982479          277  76561197970982479   
1        76561197970982479          277  76561197970982479   
2        76561197970982479          277  76561197970982479   
3        76561197970982479          277  76561197970982479   
4        76561197970982479          277  76561197970982479   
...                    ...          ...                ...   
5170010  76561198329548331            7  76561198329548331   
5170011  76561198329548331            7  76561198329548331   
5170012  76561198329548331            7  76561198329548331   
5170013  76561198329548331            7  76561198329548331   
5170014  edward_tremethick            0  76561198331598578   

                                                  user_url item_id  \
0        http://steamcommunity.com/profiles/76561197970...      10   
1        http://steamcommunity.com/profiles/76561197970...      20   
2        http://steamcommunity.com/profiles/7

In [3]:
#Eliminar las columnas que se consideraron innecesarias para el desarrollo del proyecto 
columns_to_drop = ['steam_id','user_url','item_name','item_count', 'playtime_2weeks'] 
data_items_clean = data_final_items.drop(columns=columns_to_drop, errors='ignore')

In [4]:
#revisar la eliminacion de las columnas
data_items_clean

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0
2,76561197970982479,277,30,7.0
3,76561197970982479,277,40,0.0
4,76561197970982479,277,50,0.0
...,...,...,...,...
5170010,76561198329548331,7,373330,0.0
5170011,76561198329548331,7,388490,3.0
5170012,76561198329548331,7,521570,4.0
5170013,76561198329548331,7,519140,3.0


In [5]:
# Encontrar las filas donde TODOS los valores son NULOS
filas_nulas= data_items_clean.isnull().all(axis=1)

#Obtener los índices donde TODAS las columnas son NULAS
indices_vacios = data_items_clean.index[filas_nulas]

# Verificar el índice máximo hasta donde TODAS las filas están VACÍAS E IMPRIMIR EL RESULTADO
if not indices_vacios.empty:
    max_indices_vacios = indices_vacios.max()
    print(f"El índice máximo hasta donde todas las filas están vacías es: {max_indices_vacios}")
else:
    print("No hay filas completamente vacías en el DataFrame.")

No hay filas completamente vacías en el DataFrame.


In [6]:
# Realizar el conteo de los valores faltantes en las columnas. IMPRIMIR EL CONTEO 
valores_faltantes_user_id = data_items_clean['user_id'].isnull().sum()
print(f"Valores faltantes en la columna 'user_id': {valores_faltantes_user_id}")

valores_faltantes_item_id = data_items_clean['item_id'].isna().sum()
print(f"Número de Valores faltantes en 'item_id': {valores_faltantes_item_id}")

valores_faltantes_playtime = data_items_clean['playtime_forever'].isna().sum()
print(f"Valores faltantes en 'playtime_forever': {valores_faltantes_playtime}")

Valores faltantes en la columna 'user_id': 0
Número de valores faltantes en 'item_id': 16806
Número de valores faltantes en 'playtime_forever': 16806


In [7]:
# Borrar filas con valores faltantes en la DATA
data_items_clean = data_items_clean.dropna()

In [8]:
#verificamos la aplicacion de DROPNA a la data
data_items_clean

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0
2,76561197970982479,277,30,7.0
3,76561197970982479,277,40,0.0
4,76561197970982479,277,50,0.0
...,...,...,...,...
5170009,76561198329548331,7,346330,0.0
5170010,76561198329548331,7,373330,0.0
5170011,76561198329548331,7,388490,3.0
5170012,76561198329548331,7,521570,4.0


In [9]:
#Contar las filas con el valor (0 ó 0.0) en la columna 'playing_time_forever'
conteo_filas_playing_time = data_items_clean['playtime_forever'].isin([0, 0.0]).sum()
print(f"Número de filas con 'playtime_forever' igual a 0 o 0.0: {conteo_filas_playing_time}")



Número de filas con 'playtime_forever' igual a 0 o 0.0: 1867963


In [10]:
#Eliminar las filas donde 'playing_time_forever' es 0 o 0.0
df_items_drop_0 = data_items_clean[~data_items_clean['playtime_forever'].isin([0, 0.0])]

In [11]:
#verificamos la eliminacion de 0, 0.00 
df_items_drop_0

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
2,76561197970982479,277,30,7.0
8,76561197970982479,277,300,4733.0
9,76561197970982479,277,240,1853.0
10,76561197970982479,277,3830,333.0
...,...,...,...,...
5170007,76561198329548331,7,304930,677.0
5170008,76561198329548331,7,227940,43.0
5170011,76561198329548331,7,388490,3.0
5170012,76561198329548331,7,521570,4.0


In [12]:
# sumamos el tiempo de juego por usuario y por item 
suma= df_items_drop_0.groupby(['user_id','item_id'], as_index=False)['playtime_forever'].sum()


In [13]:
#verificamos la data
suma

,user_id,item_id,playtime_forever
0,--000--,104700,461.0
1,--000--,104900,63.0
2,--000--,105600,1172.0
3,--000--,113400,2763.0
4,--000--,1250,2949.0
...,...,...,...
3246347,zzzmidmiss,55040,69.0
3246348,zzzmidmiss,55230,973.0
3246349,zzzmidmiss,91310,146.0
3246350,zzzmidmiss,9480,7.0


In [14]:
#convertimos a INT para poder guardar en parquet
suma['playtime_forever'] = pd.to_numeric(suma['playtime_forever'], errors='coerce').astype(int)

In [15]:
# Guardamos el DataFrame en un archivo Parquet
suma.to_parquet('data_items.parquet', index=False)


print("El archivo Parquet ha sido guardado exitosamente.")

El archivo Parquet ha sido guardado exitosamente.
